## Download

In [4]:
!pip install pyodbc


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Librerias

In [5]:
import pyodbc
import pandas as pd
import numpy as np
import os
import logging

## Prueba: consulta SQL en la misma query

In [6]:
# # Configuración de logging
# logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# # Conexión a **Azure SQL**
# AZURE_SERVER = 'uaxmathfis.database.windows.net'
# AZURE_DATABASE = 'usecases'
# AZURE_DRIVER = '{ODBC Driver 17 for SQL Server}'

# azure_conn_str = f"DRIVER={AZURE_DRIVER};SERVER={AZURE_SERVER};DATABASE={AZURE_DATABASE};Authentication=ActiveDirectoryInteractive"

# # Conexión a **SQL Server LOCAL**
# LOCAL_SERVER = 'localhost'
# LOCAL_DATABASE = 'master'
# LOCAL_DRIVER = '{ODBC Driver 17 for SQL Server}'

# local_conn_str = f"DRIVER={LOCAL_DRIVER};SERVER={LOCAL_SERVER};DATABASE={LOCAL_DATABASE};Trusted_Connection=yes;TrustServerCertificate=yes"

# # Ruta a la carpeta con los archivos SQL
# SQL_FOLDER_PATH = 'src/sql/'
 
# #  Consulta SQL en Azure SQL
# SQL_QUERY = """
#   SELECT
#     Customer_ID,
#     Edad,
#     Fecha_nacimiento,
#     GENERO,
#     CAST(codigopostalid AS INT) AS CP,
#     poblacion,
#     provincia,
#     STATUS_SOCIAL,
#     [RENTA_MEDIA_ESTIMADA],
#     [ENCUESTA_ZONA_CLIENTE_VENTA],
#     [ENCUESTA_CLIENTE_ZONA_TALLER],
#     [A], [B], [C], [D], [E], [F], [G], [H], [I], [J], [K], [U2],
#     [Max_Mosaic_G], [Max_Mosaic2], [Renta_Media], [F2], [Mosaic_number]
#   FROM
#     [DATAEX].[003_clientes] cliente
#   LEFT JOIN  
#     [DATAEX].[005_cp] cp ON cliente.CODIGO_POSTAL = cp.CP
#   LEFT JOIN
#     [DATAEX].[019_mosaic] mosaic ON try_cast(cp.codigopostalid AS INT) = try_cast(mosaic.CP AS INT)
# """
 
# # 🔹 Nombre de la tabla en SQL Server Local
# NEW_TABLE_NAME = "Dim_customer"
 
# try:
#     #  Conectar a Azure SQL
#     print(f"Conectando a Azure SQL...")
#     conn_azure = pyodbc.connect(azure_conn_str)
   
#     # 🔹 Ejecutar la consulta en Azure SQL
#     print(f"Ejecutando consulta en Azure SQL...")
#     df = pd.read_sql(SQL_QUERY, conn_azure)
 
#     if df.empty:
#         print(f" La consulta no devolvió resultados. No se creará la tabla en SQL Server Local.")
#     else:
#         print(f"   - Datos extraídos: {df.shape[0]} filas")
 
#         #  Convertir NaN en columnas numéricas a 0
#         df = df.fillna(0)
 
#         #  Convertir valores numéricos problemáticos
#         for col in df.select_dtypes(include=['float64']).columns:
#             df[col] = df[col].astype(np.float32)  # Reducir precisión
       
#         for col in df.select_dtypes(include=['int64']).columns:
#             df[col] = df[col].astype(np.int32)  # Evitar valores fuera de rango
       
#         #  Conectar a SQL Server Local
#         print(f"Conectando a SQL Server Local...")
#         conn_local = pyodbc.connect(local_conn_str)
       
#         with conn_local.cursor() as cursor:
#             # 🔹 Eliminar la tabla si ya existe
#             drop_table_sql = f"DROP TABLE IF EXISTS {NEW_TABLE_NAME}"
#             cursor.execute(drop_table_sql)
#             conn_local.commit()
#             print(f"   - Tabla eliminada si existía.")
 
#             # 🔹 Crear la tabla en SQL Server Local con tipos de datos ajustados
#             columns_sql = []
#             for col in df.columns:
#                 col_type = df[col].dtype
#                 if np.issubdtype(col_type, np.float32) or np.issubdtype(col_type, np.float64):
#                     columns_sql.append(f'[{col}] FLOAT')
#                 elif np.issubdtype(col_type, np.int32) or np.issubdtype(col_type, np.int64):
#                     columns_sql.append(f'[{col}] INT')
#                 else:
#                     columns_sql.append(f'[{col}] NVARCHAR(255)')
 
#             create_table_sql = f"CREATE TABLE {NEW_TABLE_NAME} ({', '.join(columns_sql)});"
#             cursor.execute(create_table_sql)
#             conn_local.commit()
#             print(f"   - Tabla {NEW_TABLE_NAME} creada correctamente en SQL Server Local.")
 
#             # Insertar los datos en SQL Server Local
#             placeholders = ', '.join(['?' for _ in df.columns])
#             insert_sql = f"INSERT INTO {NEW_TABLE_NAME} VALUES ({placeholders})"
 
#             cursor.fast_executemany = True
#             cursor.executemany(insert_sql, df.values.tolist())
#             conn_local.commit()
 
#             print(f"   - {df.shape[0]} filas insertadas en {NEW_TABLE_NAME}.")
 
# except Exception as e:
#     print(f" Error: {e}")
 
# finally:
#     if 'conn_azure' in locals():
#         conn_azure.close()
#     if 'conn_local' in locals():
#         conn_local.close()

# Consulta SQL desde sql_files

In [7]:
# Configuración de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Conexión a **Azure SQL**
AZURE_SERVER = 'uaxmathfis.database.windows.net'
AZURE_DATABASE = 'usecases'
AZURE_DRIVER = '{ODBC Driver 17 for SQL Server}'

azure_conn_str = f"DRIVER={AZURE_DRIVER};SERVER={AZURE_SERVER};DATABASE={AZURE_DATABASE};Authentication=ActiveDirectoryInteractive"

# Conexión a **SQL Server LOCAL**
LOCAL_SERVER = 'localhost'
LOCAL_DATABASE = 'master'
LOCAL_DRIVER = '{ODBC Driver 17 for SQL Server}'

local_conn_str = f"DRIVER={LOCAL_DRIVER};SERVER={LOCAL_SERVER};DATABASE={LOCAL_DATABASE};Trusted_Connection=yes;TrustServerCertificate=yes"

# Ruta a la carpeta con los archivos SQL
SQL_FOLDER_PATH = 'src/sql/'

# Nombres de los archivos SQL
sql_files = [
    'Dim_customer.sql',
    'Dim_geo.sql',
    'Dim_product.sql',
    'Dim_t.sql',
    'Fact.sql'
]

def execute_sql_file(conn, sql_file_path):
    """Ejecuta un archivo SQL y devuelve un DataFrame con los resultados."""
    try:
        with open(sql_file_path, 'r', encoding='utf-8') as file:
            sql_query = file.read()
        df = pd.read_sql(sql_query, conn)
        return df if df is not None else pd.DataFrame()  # Devuelve un DataFrame vacío si no hay resultados
    except Exception as e:
        logging.error(f"Error ejecutando {sql_file_path}: {e}")
        return pd.DataFrame()  # Devuelve un DataFrame vacío en caso de error

def create_table(conn, table_name, df):
    """Crea una tabla en la base de datos local basada en el DataFrame."""
    if df is None or df.empty:
        logging.warning(f"El DataFrame para {table_name} está vacío o no es válido.")
        return

    with conn.cursor() as cursor:
        # Eliminar la tabla si ya existe
        drop_table_sql = f"DROP TABLE IF EXISTS {table_name}"
        cursor.execute(drop_table_sql)
        conn.commit()
        logging.info(f"Tabla {table_name} eliminada si existía.")

        # Crear la tabla en SQL Server Local con tipos de datos ajustados
        columns_sql = []
        for col in df.columns:
            col_type = df[col].dtype  # Acceder al dtype de la columna
            if np.issubdtype(col_type, np.float32) or np.issubdtype(col_type, np.float64):
                columns_sql.append(f'[{col}] FLOAT')
            elif np.issubdtype(col_type, np.int32) or np.issubdtype(col_type, np.int64):
                columns_sql.append(f'[{col}] INT')
            else:
                columns_sql.append(f'[{col}] NVARCHAR(255)')
                
        create_table_sql = f"""
        CREATE TABLE {table_name} (
            {', '.join(columns_sql)}
        );
        """
        cursor.execute(create_table_sql)
        conn.commit()
        logging.info(f"Tabla {table_name} creada correctamente en SQL Server Local.")

def insert_data(conn, table_name, df):
    """Inserta los datos del DataFrame en la tabla de la base de datos local."""
    if df is None or df.empty:
        logging.warning(f"No hay datos para insertar en {table_name}.")
        return

    with conn.cursor() as cursor:
        placeholders = ', '.join(['?' for _ in df.columns])
        insert_sql = f"INSERT INTO {table_name} VALUES ({placeholders})"
        cursor.fast_executemany = True
        cursor.executemany(insert_sql, df.values.tolist())
        conn.commit()
        logging.info(f"{df.shape[0]} filas insertadas en {table_name}.")

def main():
    try:
        # Conectar a Azure SQL
        logging.info("Conectando a Azure SQL...")
        conn_azure = pyodbc.connect(azure_conn_str)

        # Conectar a SQL Server Local
        logging.info("Conectando a SQL Server Local...")
        conn_local = pyodbc.connect(local_conn_str)

        for sql_file in sql_files:
            try:
                sql_file_path = os.path.join(SQL_FOLDER_PATH, sql_file)
                logging.info(f"Ejecutando {sql_file} en Azure SQL...")

                # Ejecutar la consulta y obtener los datos
                df = execute_sql_file(conn_azure, sql_file_path)

                if df.empty:
                    logging.warning(f"La consulta no devolvió resultados para {sql_file}. No se creará la tabla en SQL Server Local.")
                    continue

                logging.info(f"Datos extraídos para {sql_file}: {df.shape[0]} filas")

                # Convertir NaN en columnas numéricas a 0
                df = df.fillna(0)

                # Convertir valores numéricos problemáticos
                for col in df.select_dtypes(include=['float64']).columns:
                    df[col] = df[col].astype(np.float32)  # Reducir precisión

                for col in df.select_dtypes(include=['int64']).columns:
                    df[col] = df[col].astype(np.int32)  # Evitar valores fuera de rango

                # Crear la tabla e insertar los datos
                table_name = sql_file.split('.')[0]
                create_table(conn_local, table_name, df)
                insert_data(conn_local, table_name, df)

            except Exception as e:
                logging.error(f"Error procesando {sql_file}: {e}")
                continue  # Continuar con el siguiente archivo

    except pyodbc.Error as e:
        logging.error(f"Error de conexión a la base de datos: {e}")
    except Exception as e:
        logging.error(f"Error inesperado: {e}")
    finally:
        if 'conn_azure' in locals():
            conn_azure.close()
        if 'conn_local' in locals():
            conn_local.close()

    logging.info("\n¡Proceso completado!")

if __name__ == "__main__":
    main()

2025-03-24 15:51:58,071 - INFO - Conectando a Azure SQL...
2025-03-24 15:51:58,497 - INFO - Conectando a SQL Server Local...
2025-03-24 15:51:58,513 - INFO - Ejecutando Dim_customer.sql en Azure SQL...
C:\Users\aleja\AppData\Local\Temp\ipykernel_21216\3684717325.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, conn)
2025-03-24 15:52:12,241 - INFO - Datos extraídos para Dim_customer.sql: 44053 filas
2025-03-24 15:52:12,306 - INFO - Tabla Dim_customer eliminada si existía.
2025-03-24 15:52:12,313 - INFO - Tabla Dim_customer creada correctamente en SQL Server Local.
2025-03-24 15:52:13,396 - INFO - 44053 filas insertadas en Dim_customer.
2025-03-24 15:52:13,396 - INFO - Ejecutando Dim_geo.sql en Azure SQL...
C:\Users\aleja\AppData\Local\Temp\ipykernel_21216\3684717325.py:35: UserWarning: pandas only sup